# 신경망 시작하기

## 신경망의 구조
- 네트워크를 구성하는 __층__
- 입력 데이터와 그에 상응하는 __타깃__
- 학습에 사용할 피드백신호를 정의하는 __손실 함수__
- 합습 진행 방식을 결정하는 __옵티마이저__

### 층 : 딥러닝의 구성단위
- 하나 이상의 텐서를 입력 받아 하나 이상의 텐서를 출력하는 데이터 처리 모듈
- 가중치 라는 층의 상태를 가짐
- 가중치는 확률적 경사 하강법에 의해 학습되는 텐서, 네트워크가 학습한 지식이 담겨있음
-


### 모델 : 층의 네트워크
- 네트워크 구조는 가설 공간을 정의
- 네트워크 구조를 선택함으로써 가능성 있는 공간(가설공가)을 입력 데이터에서 출력 데이터로 매핑하는 일련의 특정 텐서 연산으로 제한
- 이런 텐서 연산에 포함된 가중치 텐서의 좋은 값을 찾아야함

### 손실 함수와 옵티마이저
- __손실 함수__ : 훈련하는 동안 최소화될 값, 주어진 문제에 대한 성공 지표
- __옵티마이저__ : 손실 함수를 기반으로 네트워크가 어떻게 업데이트 될 지 결정, 특정 종류의 확률적 경사 하강법을 구현
여러개의 출력을 내는 신경망은 여러개의 손실함수를 가질 수 있음, 그러나 형사 하강법 과정은 하나의 스칼라 손실 값을 기준으로함. 따라서 모든 손실을 평균냄

| 문제       | 올바른 손실함수    |
|----------|-------------|
| 이진 분류 문제 | 이진 크로스엔트로피  |
| 다중 분류 문제 | 범주형 크로스엔트로피 |
| 회귀문제     | 평균제곱오차      |
| 시퀀스 학습 문제 | CTC |

### 케라스 빠르게 둘러보기
| Sequential 클래스(가장 자주 사용하는 구조인 층을 순서대로 쌓아올린 네트워크)
| keras를 import 할 지 tensorflow.keras를 import 할 지
https://89douner.tistory.com/279

In [1]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))

In [3]:
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)

 함수형 API를 만들면 모델이 처리할 데이터 텐서를 만들고 마치 이를 함수 처럼 텐서에 층을 적용함

In [5]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),
              loss='mse',
              metrics=['accuracy'])

model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)

## 영화 리뷰 분류 : 이진 분류 예제

### 데이터셋 로드하기

In [7]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 2s 0us/step


In [9]:
train_labels[0]

1

In [10]:
max([max(sequence) for sequence in train_data])

9999

- 영어로 어떻게 바꾸는 지 보기

In [20]:
def get_decoded_review(index):
    word_index = imdb.get_word_index()
    reverse_word_index = dict([value, key] for (key, value) in word_index.items()) # 정수 인덱스와 단어를 매핑하도록 뒤집음
    decoded_review = ' '.join([reverse_word_index.get(i -3, '?') for i in train_data[index]])
    return decoded_review


In [21]:
get_decoded_review(3)

"? the ? ? at storytelling the traditional sort many years after the event i can still see in my ? eye an elderly lady my friend's mother retelling the battle of ? she makes the characters come alive her passion is that of an eye witness one to the events on the ? heath a mile or so from where she lives br br of course it happened many years before she was born but you wouldn't guess from the way she tells it the same story is told in bars the length and ? of scotland as i discussed it with a friend one night in ? a local cut in to give his version the discussion continued to closing time br br stories passed down like this become part of our being who doesn't remember the stories our parents told us when we were children they become our invisible world and as we grow older they maybe still serve as inspiration or as an emotional ? fact and fiction blend with ? role models warning stories ? magic and mystery br br my name is ? like my grandfather and his grandfather before him our prot

### 데이터 준비
- 신경망에 숫자 리스트를 주입할 수 없다 -> 텐서로 변경 필요
    - 같은 길이가 되도록 리스트에 패딩을 추가하고 (samples, sequence_length) 크기의 정수 텐서로 변환, 그 다음 이 정수 텐서를 다룰 수 있는 층을 신경망의 첫번째 층으로 사용
    - 리스트를 원-핫 인코딩 하여 0과 1의 벡터로 변환. 예를 들어 시퀀스 [3, 5]를 인덱스 3과 5의 위치는 1이고 그 외는 모두 0인 10000차원의 벡터로 변환, 그 다음 부동 소수 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫 번째 층으로 사용

- 두번째 방식으로 사용하기

In [31]:
# 이해안되서 프린트 찍어봄, 이따 다시 보기

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) # 크기가 (len(sequence), dimension)이고 모든 원소가 0인 행렬 생성
    for i, sequence in enumerate(sequences):
        print(i)
        print()
        print(sequence)
        print()
        print()
        print()
        results[i, sequence] = 1.
        print(results)
        print('='*20)
    return results

x_train = vectorize_sequences(train_data)

0

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]



[[0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0.

KeyboardInterrupt: 

In [38]:
print(x_train[10])

0.0
